# 1. Ensemble of Random Forests with Resampling

In this notebook we use an enseble of random forest to predict the customer satisfaction. One of the more poignant characteristics of the data is the imbalance between the target classes, only ~4.5% of the observations are labeled as unsatisfied. To deal with this issue we undersample the majority class. The procedure is as follows (for details see train_forests.py):

- Split the training data according to the target variable: happy and unhappy customers.

- Randomly pick a specified fraction of the unhappy data.

- Merge the previous observations with a equally sized random sample of happy observations.

This gives us a balanced data set on which to train our random forests.

Remark: The metric that we use for evaluation is the area under the ROC curve.

In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
# We import custom utility functions for data processing and random forest training
from process_data import process, create_submission, drop_columns
from train_forests import trainForests, mean_ensemble

In [4]:
#data = pd.read_csv('data/train_saldo.csv')
data = pd.read_csv('data/train_extended_saldo.csv')

In [5]:
train, test = train_test_split(data, test_size = 0.2, random_state = 42)

In [6]:
X_train, Y_train = train.ix[:,:-1], train['TARGET']

In [ ]:
X_test, Y_test = test.ix[:,:-1], test['TARGET']

## 1.2 Single Forest Classifier

To have a better understanding of the problem, we examine the performance of a single random forest classifier.

In [ ]:
rf = RandomForestClassifier(n_estimators = 150)
rf.fit(X_train, Y_train)
rf_bal = RandomForestClassifier(n_estimators = 150, class_weight='balanced')
rf_bal.fit(X_train, Y_train)
# custom undersambpling (see train_forests.py)
rf_custom = trainForests(train, n_trees = 150)[0]

In [ ]:
# rf
Y_prob = pd.DataFrame(rf.predict_proba(X_test))[1]
print("rf_score is {}".format(roc_auc_score(Y_test, Y_prob)))
# rf_bal
Y_prob = pd.DataFrame(rf_bal.predict_proba(X_test))[1]
print("rf_bal score is {}".format(roc_auc_score(Y_test, Y_prob)))
# rf_custom
Y_prob = pd.DataFrame(rf_custom.predict_proba(X_test))[1]
print("rf_custom score is {}".format(roc_auc_score(Y_test, Y_prob)))

From this we see that the built-in parameter to deal with imbalanced data sets, 'class_weight', is not really useful. However, our custom resampling really makes a difference.

## 1.3 Feature Importance

It is worthwile to try and understand what are the most important features according to a random forest classifier. One may even consider to train the final model using only some of the top features.

In [ ]:
def feature_importance(n_trees, data):
    rf = RandomForestClassifier(n_trees)
    rf.fit(data.ix[:,:-1], data.ix[:,-1])
    fimp = rf.feature_importances_
    results = {}
    for idx, name in enumerate(data.ix[:,:-1].columns):
        results[name] = fimp[idx]
    return results

In [ ]:
# We can use an initial random forest to take a look at important features
if True: # Set to 'False' to skip this step as it takes some time.
    threshold = 0.01
    rank = feature_importance(300, data)
    print("The rank of the features is as follows:")
    flag = 1
    count = 0
    for a in sorted(rank.keys(), key=rank.get)[::-1]:
        aux = rank[a]
        count+= 1
        if aux < threshold and flag:
            flag = 0
            print('---'*5)
            print('There are {} features above {} '.format(count, threshold))
            print('---'*5)
        print(a, '-->', rank[a])

**Important Remark**: This backs up our data analysis. Notice how all the new features that we created are on the top of the importance rank.

## 1.4 Ensemble of Random Forests

In the final part of the model, we train N_forest-many random forest with n_trees-many trees each. Ultimately, the prediction of the probability of being unsatisfied (TARGET = 0) is made as the geometric mean of the individual predictions.

The basic parameters in the model are:
- N_forest: number of random forests to train.
- n_trees: number of trees for each forest.
- a: the fraction of the TARGET == 1 class to be used in the training of each forest.
- w: weight of the TARGET == 0 class in the training set.

### Cross Validation

We perform a grid-search to find the best parameters, we obtain a=0.25, w=1, N_forest=90, n_trees=200.

In [ ]:
cv_data = pd.read_csv("cross-validation/cross-val.csv")

In [ ]:
cv_data.describe()

In [ ]:
cv_data[cv_data['auc_roc'] > 0.835]

We compromise with N_forest = 30 and n_trees = 200 for the sake of
computer performance.

### Model Training

In [ ]:
a = 0.25 # It can also be >1 to oversample the minority class (1 ie. unhappy)
w = 1  # Weight of the majority class in the final sample.
N_forest = 30  # Number of forests to train
n_trees = 200  # Number of trees in each forest

In [ ]:
rfs = trainForests(train, a, w, N_forest, n_trees)

In [ ]:
# mean_ensemble takes the predicted probabilities of each random forest and computes the geometric mean. See train_forests.py for more details
Y_tot = mean_ensemble(rfs, X_test)

In [ ]:
temp = roc_auc_score(Y_test,Y_tot['geometric'])
rf_classifiers = [rfs, temp]
print(temp)

In [ ]:
pickle.dump(rf_classifiers, open("models/rf_classifier_param1.dat", "wb"))

For comparison, we print the score of the prediction on the training set.

In [ ]:
Y_prob = mean_ensemble(rfs, X_train)

In [ ]:
roc_auc_score(Y_train, Y_prob["geometric"])

This indicates that some overfitting might be going on. We are going to try and fix that.

#### Help with overfitting: 'min_samples_leaf'

There is another parameter that we can use to prevent overfitting, 'min_samples_leaf'. This is the number of samples that we require each leaf to have in each decision tree of the random forest. The higher the less prone to over fitting. We run a token experiment to see how scores change.

In [ ]:
for msl in range(1, 11):
    rf = RandomForestClassifier(n_estimators = 150, min_samples_leaf = msl)
    rf.fit(train.ix[:,:-1],train['TARGET'])
    temp = pd.DataFrame(rf.predict_proba(X_test))[1]
    print("msl={} --> {}".format(msl,roc_auc_score(Y_test, temp)))

A good choice for the parameter is msl = 3. Let's see whether it improves our model.

In [ ]:
rfs = trainForests(train, a, w, N_forest, n_trees, msl=3)
print(rfs[-1].get_params)
Y_tot = mean_ensemble(rfs, X_test)
temp = roc_auc_score(Y_test,Y_tot['geometric'])
rf_classifiers.append([rfs, temp])
print("Test error = {}".format(temp))
Y_prob = mean_ensemble(rfs, X_train)
print("Training error = {}".format(roc_auc_score(Y_train, Y_prob["geometric"])))

## Meta Ensemble

Now we are going to use the random forest classifiers trained above to create a meta predictor.

In [ ]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
train.head()

In [ ]:
X_train, Y_train = train.ix[:,:-1], train['TARGET']

In [ ]:
X_train.head()

In [ ]:
for rf in rfs:
    temp = rf.predict(train.ix[:,:-1])
    temp = pd.DataFrame(temp)
    X_train = pd.concat([X_train, temp], axis=1)

In [ ]:
train_meta = pd.concat([X_train, train['TARGET']], axis=1)

In [ ]:
train_meta.head()

In [ ]:
X_test, Y_test = test.ix[:,:-1], test['TARGET']

In [ ]:
for rf in rfs:
    temp = rf.predict(test.ix[:,:-1])
    temp = pd.DataFrame(temp)
    X_test = pd.concat([X_test, temp], axis=1)

In [ ]:
Y_test = test['TARGET']

In [ ]:
X_test.head()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# random regularization
cv = {}

In [ ]:
for _ in range(20):
    n_trees = np.random.randint(50,300)
    lg = RandomForestClassifier(n_estimators = n_trees)
    lg.fit(X_train, Y_train)
    cv[n_trees] = roc_auc_score(Y_test, lg.predict(X_test))
    print(cv[n_trees])

In [ ]:
for key in sorted(cv.keys(), key=cv.get)[::-1]:
    print(key,cv[key])

### Score analysis

In [ ]:
X_test, Y_test = test.ix[:,:-1], test.ix[:,-1]
n = 100
a = 0.25
w = 1
N_forest = 5
n_trees = 5

In [ ]:
scores = []
for _ in range(n):
    rfs = trainForests(train, a, w, N_forest,n_trees)
    Y_prob = mean_ensemble(rfs, X_test)
    scores.append(roc_auc_score(Y_test,Y_prob))
scores = pd.DataFrame(scores)

In [ ]:
scores.describe()

In [ ]:
plt.title("Distribution of scores")
plt.hist(scores)
plt.show()

In [ ]:
# If desired, transform probabilities into class labels.
def threshold(Y_prob, threshold = 0.5):
    result = []
    for y in Y_prob:
        if y <= threshold:
            result.append(0)
        else:
            result.append(1)
    return result

In [ ]:
# Evaluate class labels
Y_pred = threshold(Y_prob, threshold = 0.5)
_ = eval_classification(test['TARGET'],Y_pred, print_results = True)

In [ ]:
# Plot feature importance
def plot_features(forest):  
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]
    n=len(indices)
    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(n), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(n), indices)
    plt.xlim([-1, n])
    plt.show()

In [ ]:
train.head()

# Create Submission

In [ ]:
# Retrain forest on the whole 'train.csv' data
rfs = trainForests(data, a, w, N_forest, n_trees, msl)

In [ ]:
test = pd.read_csv('data/test.csv')
test_id = test.ix[:,'ID'].values
process(test)

In [ ]:
Y_prob = mean_ensemble(rfs,test)

In [ ]:
create_submission(test_id, Y_prob['geometric'])

## Ensemble RF and XGBOOST

In [ ]:
Y_boost = pd.read_csv('../Kaggle_Santander-master/simplexgbtest.csv')

In [ ]:
Y_boost.head()

In [ ]:
Y_rf = pd.read_csv('submissions/rforest_ensemble2.csv')

In [ ]:
Y_rf.head()

In [ ]:
Y_prob = pd.concat([Y_boost,Y_rf.ix[:,'TARGET']], axis=1, ignore_index=True)

In [ ]:
Y_prob.rename(columns ={0:'ID', 1:'xgb', 2: 'rfe' }, inplace = True)

In [ ]:
# geometric mean ensemble
l = 2 #number of predictors to ensemble
temp = Y_prob.ix[:,1:].product(axis=1)
temp = temp.apply(lambda x: np.power(x, 1./l))
Y_prob['geometric'] = temp

In [ ]:
# arithmetic mean ensemble
l = 2 #number of predictors to ensemble
temp = Y_prob[['xgb', 'rfe']].mean(axis=1)
temp = temp.apply(lambda x: np.power(x, 1./l))
Y_prob['arithmetic'] = temp

In [ ]:
# difference column
temp = Y_prob['xgb'] - Y_prob['rfe']
Y_prob['xgb - rfe'] = temp

In [ ]:
# difference column
temp = Y_prob['geometric'] - Y_prob['arithmetic']
Y_prob['geo - ari'] = temp

In [ ]:
Y_prob.head()

In [ ]:
plt.title('Differences between XGB and RFE')
plt.hist(Y_prob['xgb - rfe'])
plt.show()

In [ ]:
plt.title('Differences between ensembles')
plt.hist(Y_prob['geo - ari'])
plt.show()

In [ ]:
create_submission(test_id, Y_prob['arithmetic'])